# Start

In [7]:
!pip3 install langchain
!pip3 install openai
!pip3 install neo4j
!pip3 install langchain_community
!pip3 install sentence_transformers
!pip3 install -qU langchain-mistralai langchain_openai
!pip3 install transformers tiktoken
!pip3 install gradio
!pip3 install mlflow

In [2]:
import os
import json
import pandas as pd

In [ ]:
file_path = 'labor_code_structure.json'

with open(file_path, 'r') as file:
    jsonData = json.load(file)

In [9]:
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

ImportError: cannot import name 'Neo4jGraph' from 'langchain.graphs' (c:\Users\oxygen\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\graphs\__init__.py)

# Importing data

In [ ]:
def sanitize(text):
    text = str(text).replace("'", "").replace('"', '').replace('{', '').replace('}', '')
    return text

In [ ]:
# Root node
root_node = "Кодекс законів про працю України"

# Create root node
query = f'''
    MERGE (root:Root {{name: "{sanitize(root_node)}"}})
'''
graph.query(query)

[]

In [ ]:
# Loop through each article and add them to the graph
for article, references in jsonData[root_node].items():
    article = sanitize(article)

    # Create article node
    query = f'''
        MERGE (root:Root {{name: "{sanitize(root_node)}"}})
        MERGE (article:Article {{name: "{article}"}})
        MERGE (root)-[:CONTAINS]->(article)
    '''
    graph.query(query)

    # Create reference nodes and relationships
    for ref in references:
        ref = sanitize(ref)
        query = f'''
            MERGE (article:Article {{name: "{article}"}})
            MERGE (reference:Reference {{id: "{ref}"}})
            MERGE (article)-[:REFERS_TO]->(reference)
        '''
        graph.query(query)

In [ ]:
# Extract related document IDs
related_document_ids = set()
# Loop through each article and collect document IDs
for article, references in jsonData[root_node].items():
    for ref in references:
        related_document_ids.add(ref)

# Count unique related document IDs
unique_count = len(related_document_ids)

# Print the count
print(f"Unique related document IDs: {unique_count}")

Unique related document IDs: 130


In [ ]:
file_path = 'the_labour_code_of_ukraine.txt'
with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()

In [ ]:
data

'Кодекс законів про працю України\n{Затверджується Законом № 322-VIII від 10.12.71 ВВР, 1971, додаток до № 50, ст. 375}\n{Із змінами, внесеними згідно з Указами Президії ВР Української РСР № 2048-08 від 18.09.73 , ВВР 1973, № 40, ст.343 № 3866-08 від 05.06.75 , ВВР 1975, № 24, ст.296 № 1616-09 від 24.12.76 , ВВР 1977, № 1, ст. 4 № 5584-09 від 17.01.80 , ВВР 1980, № 5, ст. 81 № 2240-10 від 29.07.81 , ВВР 1981, № 32, ст.513 № 2957-10 від 30.12.81 , ВВР 1982, № 2, ст. 23 № 4617-10 від 24.01.83 , ВВР 1983, № 6, ст. 87 № 6237-10 від 21.12.83 , ВВР 1984, № 1, ст. 3 № 8474-10 від 27.02.85 , ВВР 1985, № 11, ст.205 № 2444-11 від 27.06.86 , ВВР 1986, № 27, ст.539 № 3546-11 від 10.02.87 , ВВР 1987, № 8, ст.149 № 4534-11 від 03.09.87 , ВВР 1987, № 37, ст.715 № 4841-11 від 30.10.87 , ВВР 1987, № 45, ст.904 № 5938-11 від 27.05.88 , ВВР 1988, № 23, ст.556 № 7543-11 від 19.05.89 , ВВР 1989, № 22, ст.235 № 9280-11 від 14.05.90 , ВВР 1990, № 22, ст.367 Законами Української РСР № 871-12 від 20.03.91 , ВВ

In [ ]:
import re
articles = re.finditer(r'Стаття \d+\s*(?:-\s*\d+)?\s*\..*?(?=\nСтаття \d+|$)', data, re.DOTALL)
article_data = {}

for article in articles:
    article_text = article.group()
    # Extract the article number and title
    article_number = re.search(r'Стаття (\d+\s*(?:-\s*\d+)?)\s*\.', article_text).group(1).replace(" ", "")
    article_data[f"Стаття {article_number}"] = article_text

In [ ]:
for name, text in article_data.items():
    sanitized_name = sanitize(name)
    sanitized_text = sanitize(text)
    query = f'''
        MATCH (a:Article {{name: "{sanitized_name}"}})
        SET a.text = "{sanitized_text}"
    '''
    graph.query(query)

adding date to reference node

In [ ]:
import re

def get_reference_nodes():
    query = "MATCH (n:Reference) RETURN n.name AS name, id(n) AS id"
    result = graph.query(query)
    return [{"id": record["id"], "name": record["name"]} for record in result]

def update_reference_node_date(node_id, date):
    query = """
    MATCH (n:Reference)
    WHERE id(n) = $id
    SET n.date = $date
    """
    graph.query(query)

def extract_dates_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Regex pattern to match document blocks and extract status lines
    doc_pattern = re.compile(r"\{START doc_id: \d+-\d+\}.*?\{END doc_id: \d+-\d+\}", re.DOTALL)
    status_pattern = re.compile(r"\{STATUS:.*?, від (\d{2}\.\d{2}\.\d{4})\}")

    doc_blocks = doc_pattern.findall(text)
    doc_dates = {}

    for block in doc_blocks:
        status_line = status_pattern.search(block)
        if status_line:
            doc_id = re.search(r"doc_id: (\d+-\d+)", block).group(1)
            date = status_line.group(1)
            doc_name = re.search(r"Документ (\d+-[A-Z]+)", block).group(1)
            doc_dates[doc_name] = date
    return doc_dates

'''
file_path = "related_documents.txt"
doc_dates = extract_dates_from_file(file_path)
reference_nodes = get_reference_nodes()
for node in reference_nodes:
    name = node["name"]
    if name in doc_dates:
        date = doc_dates[name]
        update_reference_node_date, node["id"], date'''

In [ ]:
reference_nodes = get_reference_nodes()

In [ ]:
for node in reference_nodes:
    name = node["name"]

In [ ]:
doc_dates = extract_dates_from_file(file_path)
doc_dates

{}

# Embedding

In [ ]:
from sentence_transformers import SentenceTransformer
from typing import List
import numpy as np

class MyEmbeddings:
        def __init__(self, model):
            self.model = SentenceTransformer(model, trust_remote_code=True)

        def normalize_vector(self, vector):
            norm = np.linalg.norm(vector)
            if norm == 0:
                return vector
            return vector / norm

        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            embeddings = [self.model.encode(t) for t in texts]
            normalized_embeddings = [self.normalize_vector(embedding).tolist() for embedding in embeddings]
            return normalized_embeddings

        def embed_query(self, text: str) -> List[float]:
            embedding = self.model.encode([text])[0]
            normalized_embedding = self.normalize_vector(embedding)
            return normalized_embedding.tolist()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
embeding_model = MyEmbeddings('lang-uk/ukr-paraphrase-multilingual-mpnet-base')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.08k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import Neo4jVector

vector_index = Neo4jVector.from_existing_graph(
    embeding_model,
    search_type="hybrid",
    node_label="Article",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

# Retrieving

In [ ]:
from langchain_core.documents.base import Document

def custom_retrieve(query: str) -> List[Document]:
    """Return documents with a score higher than 0.8"""
    res = vector_index.similarity_search_with_relevance_scores(query, k=300)
    filtered_docs = [doc for doc, score in res if score > 0.8]
    return filtered_docs[:50]

In [ ]:
custom_ret = custom_retrieve("на яких підземних роботах дозволено працювати жінкам?")
for d in custom_ret:
  first_line = d.page_content.split('.')[0]
  print(first_line)

In [ ]:
len(custom_ret)

78

# RAG chain

In [ ]:
from transformers import AutoTokenizer, GPT2Tokenizer
import tiktoken

class MistralTokenizer:
    def __init__(self, name: str):
        self.model = AutoTokenizer.from_pretrained(name)

    def encode(self, query: str):
      return self.model(query)['input_ids']


mistral_tokenizer = MistralTokenizer("mistralai/Mistral-7B-Instruct-v0.3")
openai_tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo-0125')

def filter_for_quota(tokenizer, texts: List[Document]):
    max_token_count = 16000
    result_texts = []
    current_token_count = 0

    for text in texts:
        text_token_count = len(tokenizer.encode(text.page_content))
        if current_token_count + text_token_count <= max_token_count:
            result_texts.append(text)
            current_token_count += text_token_count
        else:
            break

    return result_texts

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
len(filter_for_quota(openai_tokenizer, custom_ret))

25

In [ ]:
from langchain_mistralai import ChatMistralAI

mistral = ChatMistralAI(model="open-mistral-7b")

In [ ]:
from langchain_openai import ChatOpenAI

gpt35turbo = ChatOpenAI(model="gpt-3.5-turbo-0125")
gpt4o = ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Here are several articles of the labor code of Ukraine. Use it to answer the question at the end.
Note that these articles are retrieved from vector store and some of them may not be as useful as others.
Be concise and not repeat legalese passages word-for-word,
because the reader has access to the full text of all the documents and can read the whole thing if they want to.
Answer in Ukrainian language.

Context: {context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

def calculate_token_length(tokenizer, text):
    return len(tokenizer.encode(text))

def format_docs(docs, tokenizer, token_limit):
    context = ""
    total_tokens = 0

    for doc in docs:
        doc_tokens = calculate_token_length(tokenizer, doc.page_content)
        if total_tokens + doc_tokens <= token_limit:
            context += doc.page_content + "\n\n"
            total_tokens += doc_tokens
        else:
            break

    return context

'query = "Які є причини для звільнення з роботи?"\n\nmodel_bundle = [{"model": gpt35turbo, \'tokenizer\': openai_tokenizer}, \n                {"model": gpt4o, \'tokenizer\': openai_tokenizer},\n                {"model": mistral, \'tokenizer\': mistral_tokenizer}]\n\nchoice = 1\n\ncurrent_llm = model_bundle[choice]["model"]\ncurrent_tokenizer = model_bundle[choice]["tokenizer"]\n\nrag_chain = (\n    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}\n    | custom_rag_prompt\n    | current_llm\n    | StrOutputParser()\n)\n\nrag_chain.invoke({\n    "context": format_docs(custom_retrieve(query), current_tokenizer, 8000),\n    "question": query\n})'

In [ ]:
import gradio as gr

model_bundle = [{"model": gpt35turbo, 'tokenizer': openai_tokenizer},
                {"model": gpt4o, 'tokenizer': openai_tokenizer},
                {"model": mistral, 'tokenizer': mistral_tokenizer}]

choice = 2

current_llm = model_bundle[choice]["model"]
current_tokenizer = model_bundle[choice]["tokenizer"]

rag_chain = (
    {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
    | custom_rag_prompt
    | current_llm
    | StrOutputParser()
)

def rag_chain_wrapper(query):
    return rag_chain.invoke({
        "context": format_docs(custom_retrieve(query), current_tokenizer, 8000),
        "question": query
    })

demo = gr.Interface(
    fn=rag_chain_wrapper,
    inputs=["text"],
    outputs=["text"],
)

demo.launch()

# Benchmark

In [ ]:
import time

def model(input_df):
    answer = []
    call_interval = 30  # 2 calls per minute = 1 call every 30 seconds
    for index, row in input_df.iterrows():
        answer.append(rag_chain_wrapper(row["questions"]))
        if (index + 1) % 2 == 0:  # Check if 2 calls have been made
            time.sleep(call_interval)

    return answer

eval_df = pd.DataFrame(
    {
        "questions": [
            "Які гарантії для працівників, обраних на виборні посади?",
            "Які мої основні права як працівника?",
            "Як довго діє трудовий договір?",
            "Який порядок вивільнення працівників?",
            "В чому суть спрощенного режиму регулювання трудових відносин?",
            "Що таке норми праці?",
            "Як обраховується зарплатня за понаднормові години?",
            "Яку матеріальну відповідальність я несу, працюючи продавцем-консультантом?",
            "Які обов'язки у працівників та роботодавців?",
            "Чи маю я право на якусь компенсацію, якщо роботодавець не видав спеціальний одяг?",
            "На яку роботу я можку влаштуватися як підліток 16 років?",
            "Як вирішити трудовий спор?",
            "Чи обов'язково мені вступати в профспілку?",
            "Скільки годин на тиждень можна працювати максимально?",
            "Кому належить відпустка через народження дитини?",
            "На яких підзмених роботах дозволено працювати жінкам?",
            "Чи переноситься вихідний день на Великдень?",
            "Чи можу я працювати вночі, якщо я жінка і маю дитину віком 2 роки?",
            "В чому різниця між колективним та трудовими договорами?",
            "Які є причини для звільнення з роботи?",
        ],
        "ground_truth": [
            "Працівники, обрані на виборні посади, мають наступні гарантії:\n\n1. **Збереження роботи**: після закінчення повноважень їм надається попередня робота (посада), а при її відсутності — інша рівноцінна робота (посада) на тому самому або іншому підприємстві за згодою працівника.\n\n2. **Захист від звільнення**: для членів виборних профспілкових органів звільнення допускається лише за попередньою згодою відповідного профспілкового органу. Звільнення з ініціативи роботодавця не допускається протягом року після закінчення строку повноважень, за винятком певних обставин (ліквідація підприємства, невідповідність посаді за станом здоров’я, порушення).\n\n3. **Соціальні пільги**: зберігаються соціальні пільги та заохочення, встановлені для інших працівників за місцем роботи. Можливі додаткові пільги за рахунок коштів підприємства, якщо це передбачено колективним договором.\n\n4. **Час для виконання громадських обов’язків**: надається вільний від роботи час для участі в громадських обов’язках, консультаціях, переговорах із збереженням середньої заробітної плати.\n\n5. **Додаткова відпустка**: надається до 6 календарних днів для профспілкового навчання з компенсацією середньої заробітної плати за рахунок профспілкової організації.",
            "Основні права працівника в Україні включають:\n\n1. Право на працю, тобто на отримання роботи з оплатою не нижче встановленого державою мінімального розміру.\n2. Право на вільний вибір професії, роду занять і роботи.\n3. Право на відпочинок, включаючи обмеження робочого дня і тижня, щорічні оплачувані відпустки.\n4. Право на здорові і безпечні умови праці.\n5. Право на гідне ставлення з боку роботодавця та інших працівників.\n6. Право на об'єднання в професійні спілки та вирішення колективних трудових конфліктів.\n7. Право на участь в управлінні підприємством, установою чи організацією.\n8. Право на матеріальне забезпечення в порядку соціального страхування у випадках старості, хвороби, реабілітації, втрати працездатності, безробіття.\n9. Право звертатися до суду для вирішення трудових спорів незалежно від характеру виконуваної роботи чи займаної посади.",
            "Трудовий договір може бути безстроковим або строковим. Безстроковий договір укладається на невизначений строк. Строковий договір укладається на визначений строк або на час виконання певної роботи. Якщо після закінчення строку трудового договору трудові відносини тривають і жодна сторона не вимагає їх припинення, договір вважається продовженим на невизначений строк.",
            "Порядок вивільнення працівників включає кілька основних кроків:\n\n1. **Попередження**: Працівників персонально попереджають про вивільнення не пізніше ніж за два місяці до звільнення.\n2. **Пропозиція іншої роботи**: Власник або уповноважений ним орган повинен запропонувати працівникові іншу роботу на тому ж підприємстві, установі або організації. Якщо відповідної роботи немає або працівник відмовляється, він може звернутися до державної служби зайнятості або працевлаштуватися самостійно.\n3. **Повідомлення державної служби зайнятості**: У разі масового вивільнення, роботодавець повинен повідомити державну службу зайнятості про заплановане вивільнення працівників і провести консультації з профспілками.\n4. **Пільги та переваги**: Перевагу на залишення на роботі мають працівники з вищою кваліфікацією і продуктивністю праці. При рівних умовах пріоритет надається певним категоріям працівників, визначеним законодавством.\n\nЦі вимоги не застосовуються до працівників, які вивільняються у зв’язку з мобілізацією або знищенням виробничих умов через бойові дії.",
            "Суть спрощеного режиму регулювання трудових відносин включає:\n\n1. Полегшені вимоги щодо оформлення документів при прийнятті на роботу та звільненні працівників, що дозволяє швидше розпочати або припинити трудові відносини.\n2. Можливість встановлення індивідуальних режимів роботи, графіків робочого часу та умов праці, що враховують потреби як працівників, так і роботодавців.\n3. Зменшення обсягу обов'язкової кадрової документації, що знижує адміністративне навантаження на роботодавців.\n4. Можливість використання електронних документів для оформлення трудових відносин, що спрощує та прискорює процеси.",
            "Норми праці — це стандарти виробітку, часу, обслуговування та чисельності, що встановлюються для працівників відповідно до досягнутого рівня техніки, технології та організації праці. Ці норми можуть бути змінені в разі впровадження нових технологій або організаційно-технічних заходів. Важливо зазначити, що високий рівень продуктивності окремого працівника не є підставою для перегляду норм праці.",
            "За понаднормові години робота оплачується по-різному в залежності від системи оплати праці:\n\n1. **Погодинна система оплати праці**: Надурочна робота оплачується в подвійному розмірі годинної ставки.\n2. **Відрядна система оплати праці**: За надурочну роботу виплачується доплата у розмірі 100% тарифної ставки працівника відповідної кваліфікації, оплата праці якого здійснюється за погодинною системою, за всі відпрацьовані надурочні години.\n\nКомпенсація надурочних робіт шляхом надання відгулу не допускається",
            "Якщо ви працюєте продавцем-консультантом, на вас може бути покладена повна матеріальна відповідальність, якщо ви підпадаєте під умови, визначені в статті 135-1. Це означає, що з вами може бути укладено письмовий договір про повну матеріальну відповідальність, якщо ваша робота пов'язана з зберіганням, обробкою, продажем або використанням цінностей, переданих вам роботодавцем. \n\nКрім того, можливе застосування колективної (бригадної) матеріальної відповідальності відповідно до статті 135-2, якщо неможливо розмежувати відповідальність кожного працівника окремо. У такому випадку відповідальність встановлюється за погодженням з профспілковим органом і укладається письмовий договір між роботодавцем і всіма членами колективу.",
            "Обов'язки працівників включають:\n\n1. Працювати чесно і сумлінно, виконувати розпорядження роботодавця, дотримуватися трудової і технологічної дисципліни (стаття 139).\n2. Виконувати вимоги нормативних актів про охорону праці, користуватися засобами захисту, проходити медичні огляди, співробітничати з роботодавцем у створенні безпечних умов праці (стаття 159).\n\nОбов'язки роботодавців включають:\n\n1. Організовувати працю працівників, створювати умови для зростання продуктивності, забезпечувати трудову дисципліну, дотримуватися законодавства про працю, запобігати мобінгу, покращувати умови праці і побуту (стаття 141).\n2. Створювати безпечні і нешкідливі умови праці, впроваджувати засоби техніки безпеки, проводити інструктаж з охорони праці і протипожежної безпеки (стаття 153).",
            "Так, ви маєте право на компенсацію. Якщо роботодавець не видав спеціальний одяг або спеціальне взуття у встановлений строк, і ви були змушені придбати їх за власні кошти, роботодавець зобов’язаний компенсувати ваші витрати.",
            "Як підліток 16 років, ти можеш влаштуватися на будь-яку роботу, яка не є важкою або не має шкідливих чи небезпечних умов праці. Ти не можеш працювати на підземних роботах або піднімати та переміщувати речі, маса яких перевищує встановлені для тебе граничні норми. Також ти не можеш працювати у нічний час, понаднормово або у вихідні дні.\n\nРобочий час для тебе буде скороченим і становитиме не більше 36 годин на тиждень. Заробітну плату ти отримуватимеш у такому ж розмірі, як і дорослі працівники за повний робочий день. Прийом на роботу можливий лише після попереднього медичного огляду, і щорічно до досягнення 21 року ти підлягатимеш обов’язковому медичному оглядові.",
            "Трудовий спір можна вирішити кількома способами:\n\n1. **Медіація**: Спір може бути врегульований шляхом медіації відповідно до Закону України \"Про медіацію\". Якщо угода, досягнута під час медіації, не виконується, сторони мають право звернутися до органів, передбачених статтею 221 Кодексу (стаття 222-1).\n\n2. **Комісія по трудових спорах**: Комісія зобов’язана розглянути спір у десятиденний строк з дня подання заяви. Розгляд проводиться у присутності працівника та представників роботодавця (стаття 226).\n\n3. **Місцеві загальні суди**: Якщо спір не можна вирішити через комісію по трудових спорах або якщо сторони не задоволені її рішенням, трудовий спір може бути розглянутий місцевими загальними судами (стаття 221).\n\n4. **Рішення про поновлення на роботі або зміну формулювання причин звільнення**: У разі незаконного звільнення або переведення на іншу роботу, працівник може бути поновлений на попередній роботі, а також може отримати середній заробіток за час вимушеного прогулу (стаття 235).",
            "Ні, вступ до профспілки не є обов'язковим. Відповідно до статті 243, громадяни України мають право на основі вільного волевиявлення створювати професійні спілки, вступати до них або виходити з них на умовах і в порядку, визначених їх статутами. Це право є добровільним і не потребує дозволу.",
            "Максимальна тривалість робочого часу в Україні не може перевищувати 40 годин на тиждень.",
            "Відпустка при народженні дитини надається таким працівникам:\n1. Чоловіку, дружина якого народила дитину.\n2. Батьку дитини, який не перебуває у зареєстрованому шлюбі з матір'ю дитини, за умови, що вони спільно проживають і мають взаємні права та обов'язки.\n3. Одній із таких осіб: бабі, діду або іншому повнолітньому родичу дитини, які фактично здійснюють догляд за дитиною, якщо мати (батько) є одинокою матір'ю (одиноким батьком).",
            "Жінкам дозволено працювати на підземних роботах, які є нефізичними або пов'язані з санітарним та побутовим обслуговуванням.",
            "Так, якщо Великдень збігається з вихідним днем, то вихідний день переноситься на наступний після святкового або неробочого дня. Це передбачено статтею 67 Кодексу законів про працю України.",
            "Відповідно до статті 55 та статті 176 Кодексу законів про працю України, залучення жінок, які мають дітей віком до трьох років, до роботи в нічний час забороняється. Тому, якщо у вас є дитина віком 2 роки, ви не можете працювати вночі.",
            "Колективний договір укладається між роботодавцем і колективом працівників (представленим профспілками або іншими обраними представниками) і регулює виробничі, трудові й соціально-економічні відносини на підприємстві (ст. 12, ст. 13). Він може включати додаткові порівняно з законодавством гарантії та пільги.\n\nТрудовий договір укладається між роботодавцем і конкретним працівником. Він визначає зобов’язання працівника виконувати певну роботу і зобов’язання роботодавця виплачувати заробітну плату та забезпечувати умови праці (ст. 21). Трудовий договір може включати умови щодо професійної кваліфікації та інші індивідуальні права та обов'язки.\n\nОтже, основна різниця полягає в тому, що колективний договір регулює колективні трудові відносини на рівні підприємства, тоді як трудовий договір регулює індивідуальні трудові відносини між роботодавцем і працівником",
            "Причини для звільнення з роботи включають:\n\n1. Повна ліквідація підприємства, установи, організації.\n2. Виявлена невідповідність працівника займаній посаді або виконуваній роботі через стан здоров'я.\n3. Вчинення працівником дій, за які законом передбачена можливість звільнення.\n4. Масове вивільнення працівників у зв'язку з ліквідацією, реорганізацією підприємств, зміною форм власності або частковим зупиненням виробництва.\n5. За попередньою згодою виборного профспілкового органу для членів виборних профспілкових органів.\n\nТакож звільнення можливе у випадках, коли працівник не виконує належним чином свої обов'язки або за власним бажанням."
        ]
    }
)


In [ ]:
eval_df['context'] = None

for idx, row in eval_df.iterrows():
    query = row['questions']
    eval_df.at[idx, 'context'] = format_docs(custom_retrieve(query), mistral_tokenizer, 8000)

In [ ]:
eval_df

,questions,ground_truth,context
0,"Які гарантії для працівників, обраних на вибор...","Працівники, обрані на виборні посади, мають на...","\ntext: Стаття 118. Гарантії для працівників, ..."
1,Які мої основні права як працівника?,Основні права працівника в Україні включають:\...,\ntext: Стаття 2. Основні трудові права праців...
2,Як довго діє трудовий договір?,Трудовий договір може бути безстроковим або ст...,\ntext: Стаття 49 - 6 . Трудовий договір в умо...
3,Який порядок вивільнення працівників?,Порядок вивільнення працівників включає кілька...,\ntext: Стаття 121. Гарантії і компенсації при...
4,В чому суть спрощенного режиму регулювання тру...,Суть спрощеного режиму регулювання трудових ві...,\ntext: Стаття 252 - 5 . Загальні принципи мат...
5,Що таке норми праці?,"Норми праці — це стандарти виробітку, часу, об...",\ntext: Стаття 85. Норми праці\nНорми праці - ...
6,Як обраховується зарплатня за понаднормові год...,За понаднормові години робота оплачується по-р...,\ntext: Стаття 106. Оплата роботи в надурочний...
7,"Яку матеріальну відповідальність я несу, працю...","Якщо ви працюєте продавцем-консультантом, на в...",\ntext: Стаття 145. Переваги і пільги для прац...
8,Які обов'язки у працівників та роботодавців?,Обов'язки працівників включають:\n\n1. Працюва...,\ntext: Стаття 145. Переваги і пільги для прац...
9,"Чи маю я право на якусь компенсацію, якщо робо...","Так, ви маєте право на компенсацію. Якщо робот...",\ntext: Стаття 164. Компенсаційні виплати за н...


In [ ]:
from mlflow.metrics.genai import EvaluationExample, relevance, answer_relevance, answer_similarity

judge = "openai:/gpt-4-turbo"
relevance_metric = relevance(judge)                 # groundedness    req context
answer_relevance_metric = answer_relevance(judge)   # relevance       req nothing
answer_similarity_metric = answer_similarity(judge) # similarity      req ground_truth

In [ ]:
import mlflow
results = mlflow.evaluate(
    model,
    eval_df,
    model_type="question-answering",
    evaluators="default",
    predictions="result",
    extra_metrics=[relevance_metric, answer_relevance_metric, answer_similarity_metric],
    evaluator_config={
        "col_mapping": {
            "inputs": "questions",
            "context": "context",
            "targets": "ground_truth"
        }
    },
)
print(results.metrics)

2024/06/14 14:11:32 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/06/14 14:19:12 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/06/14 14:19:12 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/06/14 14:19:12 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.
2024/06/14 14:19:12 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2024/06/14 14:19:12 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'flesch_kincaid_grade_level' because it returned None.
2024/06/14 14:19:12 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skippi

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

2024/06/14 14:19:27 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/06/14 14:19:27 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.
2024/06/14 14:19:27 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2024/06/14 14:19:27 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'flesch_kincaid_grade_level' because it returned None.
2024/06/14 14:19:27 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2024/06/14 14:19:27 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'ari_grade_level' because i

  0%|          | 0/20 [00:00<?, ?it/s]

2024/06/14 14:20:31 WARNING mlflow.openai.api_request_parallel_processor: Retrying for request failed with rate limit.
2024/06/14 14:20:31 WARNING mlflow.openai.api_request_parallel_processor: Retrying for request failed with rate limit.
2024/06/14 14:20:31 WARNING mlflow.openai.api_request_parallel_processor: Retrying for request failed with rate limit.
2024/06/14 14:20:31 WARNING mlflow.openai.api_request_parallel_processor: Retrying for request failed with rate limit.
2024/06/14 14:20:31 WARNING mlflow.openai.api_request_parallel_processor: Retrying for request failed with rate limit.
2024/06/14 14:20:31 WARNING mlflow.openai.api_request_parallel_processor: Retrying for request failed with rate limit.
2024/06/14 14:20:34 WARNING mlflow.openai.api_request_parallel_processor: Retrying for request failed with rate limit.
2024/06/14 14:20:35 WARNING mlflow.openai.api_request_parallel_processor: Retrying for request failed with rate limit.
2024/06/14 14:21:21 WARNING mlflow.openai.api_re

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

{'exact_match/v1': 0.0, 'relevance/v1/mean': 4.3, 'relevance/v1/variance': 0.21000000000000002, 'relevance/v1/p90': 5.0, 'answer_relevance/v1/mean': 5.0, 'answer_relevance/v1/variance': 0.0, 'answer_relevance/v1/p90': 5.0, 'answer_similarity/v1/mean': 3.95, 'answer_similarity/v1/variance': 1.0474999999999999, 'answer_similarity/v1/p90': 5.0}


In [ ]:
results.tables["eval_results_table"].to_csv("mistral-7b-eval.csv", index=False)